
# 🌍 Manual TOA Reflectance from Landsat 8

## 🎯 Learning Objectives:
- Load Landsat 8 Level-1 data from Earth Engine
- Understand the process of converting DN values to radiance
- Compute Top-of-Atmosphere (TOA) reflectance from radiance
- Visualize results on an interactive map using `geemap`

---

## 📝 Your Tasks:
1. Load a Landsat 8 Level-1 image over Kraków in July 2021.
2. From metadata, extract the scaling coefficients for each band (RADIANCE_MULT, RADIANCE_ADD).
3. Convert DN to Radiance using the linear formula.
4. Use the ESUN values and solar elevation to convert Radiance to TOA Reflectance.
5. Visualize both the original DN RGB and your TOA Reflectance RGB image.
6. (Optional) Try using a different location or time range.


In [2]:
import ee
import geemap
#ee.Authenticate()
ee.Initialize(project='ee-dabrowaremote')


## 🗺️ Step 1: Define Area of Interest (AOI)

Use Kraków (Poland) as your starting point. You can define this using a point geometry.


In [ ]:
#aoi = ee.Geometry.Point([20, 50])

In [3]:
aoi = ee.Geometry.Polygon([
    [[19.851, 50.064], [19.951, 50.064], [19.951, 50.014], [19.851, 50.014], [19.851, 50.064]]
])


## 📦 Step 2: Load Landsat 8 Level-1 Image (Collection 2)

Load the `LANDSAT/LC08/C02/T1_L1` collection, filter for July 2021, and select the image with the least cloud cover.


In [4]:
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2021-07-01', '2021-07-31') \
    .map(lambda image: image.clip(aoi)) \
    .sort('CLOUD_COVER') \
    .first()

#print(landsat_collection.getInfo())
landsat

In [6]:
Map = geemap.Map()
Map.centerObject(aoi, 10)  # Ustawienie widoku na obszar zainteresowania
Map.addLayer(landsat, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 30000}, 'Landsat 8 Image')
#Map


## 🔢 Step 3: Convert DN to Radiance

Use the formula:

```
Radiance = M_L * DN + A_L
```

Where:
- `M_L` is `RADIANCE_MULT_BAND_X` from metadata
- `A_L` is `RADIANCE_ADD_BAND_X`
- DN is the digital number of each band

💡 Tip: You can use `image.get('RADIANCE_MULT_BAND_4')` to retrieve the multiplier for band 4.


In [10]:
# Retrieve radiance multiplier and addend for band 5
m_l_B5 = landsat.get('RADIANCE_MULT_BAND_5').getInfo()
a_l_B5 = landsat.get('RADIANCE_ADD_BAND_5').getInfo()

# Retrieve radiance multiplier and addend for band 4
m_l_B4 = landsat.get('RADIANCE_MULT_BAND_4').getInfo()
a_l_B4 = landsat.get('RADIANCE_ADD_BAND_4').getInfo()

# Retrieve radiance multiplier and addend for band 3
m_l_B3 = landsat.get('RADIANCE_MULT_BAND_3').getInfo()
a_l_B3 = landsat.get('RADIANCE_ADD_BAND_3').getInfo()

# Retrieve radiance multiplier and addend for band 2
m_l_B2 = landsat.get('RADIANCE_MULT_BAND_2').getInfo()
a_l_B2 = landsat.get('RADIANCE_ADD_BAND_2').getInfo()

# Calculate radiance for band 5
radiance_B5 = landsat.select('SR_B5').multiply(m_l_B5).add(a_l_B5)

# Calculate radiance for band 4
radiance_B4 = landsat.select('SR_B4').multiply(m_l_B4).add(a_l_B4)
# Calculate radiance for band 3
radiance_B3 = landsat.select('SR_B3').multiply(m_l_B3).add(a_l_B3)
# Calculate radiance for band 2
radiance_B2 = landsat.select('SR_B2').multiply(m_l_B2).add(a_l_B2)

# Add the radiance layer to the map for visualization
#Map.addLayer(radiance_B4, {'min': 0, 'max': 100, 'palette': ['blue', 'green', 'red']}, 'Radiance Band 4')
#Map



## 📊 Step 4: Convert Radiance to TOA Reflectance

Use the following equation from the USGS Landsat Handbook:

```
TOA Reflectance = (π * L_λ * d²) / (ESUN * cos(θ_s))
```

Where:
- `L_λ` = radiance
- `d` = Earth–Sun distance in AU (assume 1.0 for simplification)
- `ESUN` = solar spectral irradiance (use constants from handbook)
- `θ_s` = solar zenith angle = 90° - sun elevation (from image metadata)

Use these ESUN values:
- B2: 2067  
- B3: 1893  
- B4: 1603  
- B5: 972


In [11]:
import math

# Constants
ESUN = {'B2': 2067, 'B3': 1893, 'B4': 1603, 'B5': 972}
solar_elevation = landsat.get('SUN_ELEVATION').getInfo()
solar_zenith = 90 - solar_elevation
cos_theta_s = math.cos(math.radians(solar_zenith))

# Calculate TOA Reflectance for each band
toa_reflectance_B2 = radiance_B2.multiply(math.pi).divide(ESUN['B2'] * cos_theta_s)
toa_reflectance_B3 = radiance_B3.multiply(math.pi).divide(ESUN['B3'] * cos_theta_s)
toa_reflectance_B4 = radiance_B4.multiply(math.pi).divide(ESUN['B4'] * cos_theta_s)
toa_reflectance_B5 = radiance_B5.multiply(math.pi).divide(ESUN['B5'] * cos_theta_s)




## 🖼️ Step 5: Visualize the Results

Use `geemap` to create an interactive map. Display:
- Original RGB from DN bands
- Your manually calculated TOA Reflectance RGB

💡 Hint: Use `.clamp(0, 0.4)` on reflectance bands for better visualization.


In [ ]:
# Visualize the original RGB from DN bands
Map2 = geemap.Map()
Map2.centerObject(aoi, 10)  # Ustawienie widoku na obszar zainteresowania
Map2.addLayer(landsat, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 30000}, 'Original DN RGB')

# Visualize the manually calculated TOA Reflectance RGB
toa_rgb = ee.Image.cat([toa_reflectance_B4, toa_reflectance_B3, toa_reflectance_B2]).clamp(0, 0.4)
Map2.addLayer(toa_rgb, {'min': 0, 'max': 0.4}, 'TOA Reflectance RGB')

# Display the map
Map2


## 🧪 Optional Extensions

- Try the same workflow on a different city or biome (e.g., desert, rainforest).
- Add NIR band to your analysis (B5).
- Compute NDVI from TOA or compare with Surface Reflectance data.
- Export the resulting reflectance image to Google Drive.
